### Libraries

In [1]:
import re
import os
import csv
import time
import unidecode
import pyperclip
import pandas as pd
from thefuzz import fuzz
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains

### Functions

In [7]:
def page_has_loaded(driver):
    page_state = driver.execute_script('return document.readyState;')
    return page_state == 'complete'

In [8]:
def get_options():
    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--start-maximized")
    #chrome_options.add_argument("--headless")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    # chrome_options.add_argument('--proxy-server=%s' % proxy)
    return chrome_options

### Selenium Initialization

* Constants

In [9]:
BASE_URL = 'https://www.google.com.br/maps/'
SELENIUM_PATH = 'C:\selenium_drivers\chromedriver.exe'
NOME_ARQ_FINAL = 'gplaces_educacao_v3'
RE_CATCH_ALL_URLS = r'((http|ftp|https):\/\/)?([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])'

* Initialization

In [11]:
ser = Service(SELENIUM_PATH)

driver = webdriver.Chrome(service=ser, options=get_options())
driver.execute_cdp_cmd('Network.setUserAgentOverride', {
        "userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'});

* Load the dataset with params for searching

In [12]:
%time df_raw = pd.read_excel(r"C:\Users\prove\Desktop\export_felipe_5.xlsx", sheet_name='Base_Input_Scraping')

CPU times: total: 516 ms
Wall time: 523 ms


* Select only the columns we need

In [16]:
df_raw['location'] = df_raw['Logradouro, Numero - Bairro, Municipio - UF, CEP']#df_raw['Cidade'] + ', ' + df_raw['UF'].astype('str')
df_raw.rename(columns={'Nome': "companyName", 'CNPJ': 'cnpj'}, inplace=True)
df_raw['location'] = df_raw['location'].fillna(' ')

df_raw['companyName'] = df_raw['companyName'].replace('LTDA - ME', '')
df_raw['companyName'] = df_raw['companyName'].replace('LTDA', '')
df_anvisa = df_raw.copy()
#df_anvisa = df_raw[df_raw['tier'].isin([6,7,8,'6d','7d','8p'])].dropna(subset=['companyName', 'location'], how='any')

### Search for places

In [18]:
# Acessa URL base
driver.get(BASE_URL)
time.sleep(3)

# Continua de onde parou
idx_continue = 0
if os.path.isfile(NOME_ARQ_FINAL+".csv"):
    df_sites = pd.read_csv(NOME_ARQ_FINAL+".csv", encoding='latin-1', dtype={'PARAM_CNPJ':str})
    try:
        idx_continue = df_anvisa[df_anvisa['cnpj'] == str(df_sites['PARAM_CNPJ'].iloc[-1])].index[0]
    except:
        idx_continue = df_anvisa[df_anvisa['companyName'] == df_sites['PARAM_NOME'].iloc[-1]].index[0]
        print('except')
        pass

print('iniciando do índice:', idx_continue)

while not page_has_loaded(driver):
    time.sleep(5)

has_header = False if not os.path.isfile(NOME_ARQ_FINAL+".csv") else True # Marca se arquivo tem header

# Começa for-loop pelos estabelecimentos
for idx, row in df_anvisa.iloc[idx_continue:].iterrows():
    param_nome = row['companyName']
    #param_website = row['website']
    param_location = row['location']
    param_cnpj = row['cnpj']

    print('query: {}'.format(param_nome + ' em ' + param_location))
    
    if '/dir/' in driver.current_url:
        driver.back()
    
    #Faz a primeira pesquisa - primeira tentativa
    test = 0
    while test < 3:
        try:
            driver.implicitly_wait(5)
            elem = driver.find_element(By.XPATH, "//input[@id='searchboxinput']")
            elem.clear()
            #consultoria em Pimentas, Guarulhos - SP, Brasil
            elem.send_keys(param_nome + ' em ' + param_location)
            #elem.send_keys(Keys.RETURN)
            driver.implicitly_wait(5)
            elem_pesquisar = driver.find_element(By.XPATH, "//button[@aria-label='Pesquisar']")
            if elem_pesquisar is not None:
                #print('click')
                elem_pesquisar.click()
            else:
                #print('enter')
                elem.send_keys(Keys.RETURN)
            break
        except:
            print('tentativa_v1', test)
            test+=1
            driver.refresh()
            time.sleep(5)
            pass
        
    
    time.sleep(3)
    gmaps_url = None
    
    #Se não encontrou, faz a segunda pesquisa - segunda tentativa
    #Dessa vez pesquisa por estabelecimento na lista que foi retornada na pesquisa
    if '/place/' not in driver.current_url:
        print('/place/ not in url...') 
        test = 0
        while test < 3:
            try:
                gmaps_url = None
                max_fuzz = 0
                
                ps = driver.page_source
                bs = BeautifulSoup(ps, 'html.parser')
                
                found_urls = bs.find_all('a', {'href': re.compile(r'(place)')})
                if found_urls:
                    for company in found_urls:
                        fuzz_ratio = fuzz.ratio(param_nome_unidecode, company.attrs['aria-label'].lower())

                        if fuzz_ratio > max_fuzz: #Seleciona estabelecimento com maior similaridade com nome do estabelecimento
                            max_fuzz = fuzz_ratio
                            gmaps_url = company.attrs['href'] 

                gmaps_url
                break
            except:
                print('tentativa_v2', test)
                driver.get(BASE_URL)
    
                driver.implicitly_wait(5)
                elem = driver.find_element(By.XPATH, "//input[@id='searchboxinput']")
                elem.clear()
                #consultoria em Pimentas, Guarulhos - SP, Brasil
                elem.send_keys('"'+param_nome+'"' + ' em ' + param_location)
                #elem.send_keys(Keys.RETURN)
                driver.implicitly_wait(5)
                elem_pesquisar = driver.find_element(By.XPATH, "//button[@aria-label='Pesquisar']")
                if elem_pesquisar is not None:
                    #print('click')
                    elem_pesquisar.click()
                else:
                    #print('enter')
                    elem.send_keys(Keys.RETURN)

                time.sleep(5)
                
                test+=1
                #driver.refresh()
                #time.sleep(5)
                pass

    if gmaps_url is not None:
        driver.get(gmaps_url)
    
    while not page_has_loaded(driver):
        time.sleep(5)
        
        
    ps = driver.page_source
    bs = BeautifulSoup(ps, 'html.parser')
    
    google_url = None
    nome_empresa = None
    
    #captura nome da empresa
    try:
        for i in re.findall(r'[\s\w-]*fontHeadlineLarge\b', str(bs.html.decode)):
            if bs.find('h1', {'class': i}) is not None:
                nome_empresa = (bs.find('h1', {'class': i}).text)
                break
    except:
        print('Exception: can\'t find the company\'s name')
        pass
    
    print('\tNome:', nome_empresa)
    print('\tCOD_ID:', param_cnpj)
    
    #captura endereço
    end_copy_paste = None
    try:
        driver.implicitly_wait(5)
        elem_copy_paste = driver.find_element(By.XPATH, "//button[@aria-label='Copiar endereço']")
        if elem_copy_paste is not None:
            back_paste = pyperclip.paste()
            elem_copy_paste.click()
            time.sleep(0.3)
            end_copy_paste = pyperclip.paste()
            pyperclip.copy(back_paste)
    except:
        print('Exception: can\'t copy the end')
        pass
    
    #captura telefone
    tel_copy_paste = None
    try:
        driver.implicitly_wait(5)
        elem_copy_paste = driver.find_element(By.XPATH, "//button[@aria-label='Copiar número de telefone']")
        if elem_copy_paste is not None:
            back_paste = pyperclip.paste()
            elem_copy_paste.click()
            time.sleep(0.3)
            tel_copy_paste = pyperclip.paste()
            pyperclip.copy(back_paste)
    except:
        print('Exception: can\'t copy the tel')
        pass
    
    #captura plus code
    plus_code_copy_paste = None
    try:
        driver.implicitly_wait(5)
        elem_copy_paste = driver.find_element(By.XPATH, "//button[@aria-label='Copiar Plus Code']")
        if elem_copy_paste is not None:
            back_paste = pyperclip.paste()
            elem_copy_paste.click()
            time.sleep(0.3)
            plus_code_copy_paste = pyperclip.paste()
            pyperclip.copy(back_paste)
    except:
        print('Exception: can\'t copy the plus code')
        pass

    #captura url
    url_copy_paste = None
    try:
        driver.implicitly_wait(5)
        elem_copy_paste = driver.find_element(By.XPATH, "//button[@aria-label='Copiar website']")
        if elem_copy_paste is not None:
            back_paste = pyperclip.paste()
            elem_copy_paste.click()
            time.sleep(0.3)
            url_copy_paste = pyperclip.paste()
            pyperclip.copy(back_paste)
    except:
        print('Exception: can\'t copy the url')
        pass


    print('\tURL (copy-paste):', url_copy_paste)
    
    rating = None
    total_ratings = None
    
    #captura ratings e total ratings
    v2_rating = re.findall(r'(\d,\d) estrela[s]{0,1}', str(bs.html.decode))
    if v2_rating is not None and v2_rating != []:
        v2_rating = v2_rating[0]
        
    v2_total_rating = bs.find('button', {'aria-label': re.compile(r'\d avaliaç[ões|ão]+')})
    if v2_total_rating is not None:
        v2_total_rating = v2_total_rating.text
    
    category = bs.find('button', {'jsaction': 'pane.rating.category'})
    if category is not None:
        category = category.text
        
    print('\t{} Rating: {} Total Ratings: {}'.format(category, v2_rating, v2_total_rating))
    
    nome_empresa_unidecode = unidecode.unidecode(nome_empresa.lower()) if nome_empresa is not None else None
    param_nome_unidecode = unidecode.unidecode(param_nome.lower())  if param_nome is not None else None
    
    d_info = {
            'PARAM_CNPJ': param_cnpj,
            'PARAM_NOME': param_nome,
            #'PARAM_WEBSITE': param_website,
            'PARAM_LOCALIZACAO': param_location,
            'gmapsCompanyName': nome_empresa,
            'gmapsCategory': category,
            #'avaliacao': rating,
            'gmapsRating': v2_rating if v2_total_rating else None,
            #'qtd_avaliacoes': total_ratings,
            'gmapsNumberOfRatings': v2_total_rating if v2_rating else None,
            #'url(regex)': url,
            'gmapsAddress': end_copy_paste,
            'gmapsTelephone': tel_copy_paste,
            'gmapsPlusCode': plus_code_copy_paste,
            'gmapsWebsite': url_copy_paste,
            #'txt': txt,
            'gmapsUrl': google_url if google_url is not None else driver.current_url,
            'gmapsCompanyNameFuzzRatio': fuzz.ratio(param_nome_unidecode, nome_empresa_unidecode) if nome_empresa is not None else 0,
            'gmapsCompanyNameFuzzPartialRatio': fuzz.partial_ratio(param_nome_unidecode, nome_empresa_unidecode) if nome_empresa is not None else 0,
            'gmapsCompanyNameFuzzTokenSort': fuzz.token_sort_ratio(param_nome_unidecode, nome_empresa_unidecode) if nome_empresa is not None else 0,
            'gmapsCompanyNameFuzzTokenSet': fuzz.token_set_ratio(param_nome_unidecode, nome_empresa_unidecode) if nome_empresa is not None else 0
    }
    
    #Salva dados em arquivo
    with open(NOME_ARQ_FINAL+".csv", "a+", newline='') as csvfile:
        w = csv.DictWriter(csvfile, d_info.keys(), delimiter=',')
        if not has_header:
            w.writeheader()
            has_header = True
        try:
            #d_info.update((k,str(d_info[k])) for k in d_info if d_info[k] is not None)
            w.writerow(d_info)
        
        except:
            #for k,v in d_info.items():
            #    if d_info.get(k, None) is not None:
            #        d_info[k] = unidecode.unidecode(d_info[k])
            
            d_info.update( (k,unidecode.unidecode(str(d_info[k]))) for k in d_info if d_info[k] is not None)
            w.writerow(d_info)
            pass
    
    time.sleep(1)

except
iniciando do índice: 261
query: COLÉGIO CENECISTA FREDERICO MICHAELSE em rua: 28 de fevereiro 100 - centro, Nova Petrópolis - RS, 95150-000
	Nome:  Colégio Cenecista Frederico Michaelsen  
	COD_ID: 262
	URL (copy-paste): http://colegionovapetropolis.cnec.br/
	Escola Rating: 4,9 Total Ratings: 8 avaliações
query: COLEGIO CENECISTA JOSE ELIAS MOREIR em Rua Coronel Francisco Gomes1290 - Bucarein, Joinville - SC, 89202-250
	Nome:  Colégio Cenecista José Elias Moreira  
	COD_ID: 263
	URL (copy-paste): http://colegios.cnec.br/eliasmoreira
	Escola Rating: 3,6 Total Ratings: 77 avaliações
query: COLÉGIO CENECISTA PEDRO DO RIO em Rua Eugênio Zanatta247 - Pedro do Rio, Petrópolis - RJ, 25750-300
	Nome:  Colégio Cenecista Pedro do Rio  
	COD_ID: 264
	URL (copy-paste): http://site.cnec.br/
	Escola particular Rating: 3,0 Total Ratings: 2 avaliações
query: COLEGIO CENECISTA PRES. KENNEDY - EDUCAÇÃO INFANTIL- ENSINO FUNDAMENTAL- MÉDIO E  PROFISSIONA em Rua Rui Barbosa541 - Centro, Campo Largo 

Exception: can't copy the url
	URL (copy-paste): None
	Escola pública Rating: 4,5 Total Ratings: 15 avaliações
query: COLEGIO ESTADUAL COSTA VIANA - ENSINO FUNDAMENTAL- MEDIO- PROFISSIONAL E NORMAL em Rua Paulino de Siqueira Cortes2685 - braga, São José dos Pinhais - PR, 83005-030
	Nome:  Costa Viana, C E-Ef M Profis N  
	COD_ID: 291
	URL (copy-paste): http://www.sjpcostaviana.seed.pr.gov.br/modules/liaise/
	Escola Rating: 4,6 Total Ratings: 65 avaliações
query: COLÉGIO ESTADUAL DARIA VIANA DE QUEIROZ em  Av Sergipe0 - ipanema, Barra do Choça - BA, 45120-000
	Nome:  Colegio Estadual Daria Viana De Queiroz  
	COD_ID: 292
	URL (copy-paste): http://www.salvador.ba.gov.br/
	Repartição pública Rating: 4,3 Total Ratings: 32 avaliações
query: COLÉGIO ESTADUAL DE 1º GRAU JOSEVALDO LIMA em  Povoado Saco do Correios/n - zona rural, Lamarão - BA, 48720-000
/place/ not in url...
	Nome:  Lamarão  
	COD_ID: 293
Exception: can't copy the end
Exception: can't copy the tel
Exception: can't copy the plu

/place/ not in url...
	Nome:  Colégio Estadual Honestino Monteiro Guimarães  
	COD_ID: 317
Exception: can't copy the tel
Exception: can't copy the url
	URL (copy-paste): None
	Escola Rating: 3,0 Total Ratings: 2 avaliações
query: COLEGIO ESTADUAL IRACEMA LEITE NADER em Rua Claudionor Cabral Franco2237 - Vila Nova, Barra Mansa - RJ, 27321-010
	Nome:  CE Iracema Leite Nader  
	COD_ID: 318
Exception: can't copy the url
	URL (copy-paste): None
	Escola pública Rating: 3,9 Total Ratings: 10 avaliações
query: COLÉGIO ESTADUAL JOÃO DAS BOTAS  em MORRO DO GAVAZZA96 - BARRA, Salvador - BA, 40140-120
	Nome:  Colégio Estadual João das Botas  
	COD_ID: 319
	URL (copy-paste): http://escolas.educacao.ba.gov.br/node/12050
	Escola pública Rating: 4,1 Total Ratings: 16 avaliações
query: COLÉGIO ESTADUAL JOÃO TAVARES MARTINS em rUA 05 S/N - VALE DO ARAGUAIA, Araguaçu - TO, 77475-000
/place/ not in url...
	Nome:  Colégio Estadual João Tavares Martins  
	COD_ID: 320
Exception: can't copy the tel
Exception:

query: COLEGIO ESTADUAL PINHEIRO DO PARANA - ENISNO FUNDAMENTAL- MÉDIO E PROFISSIONA em Rua Daniel Cesário Pereira559 - Santa Felicidade, Curitiba - PR, 82410-180
	Nome:  Pinheiro Do Parana, C E-Ef M Profis  
	COD_ID: 344
Exception: can't copy the url
	URL (copy-paste): None
	Escola Rating: 4,4 Total Ratings: 19 avaliações
query: COLEGIO ESTADUAL POLIVALENTE - ENSINO FUNDAMENTAL- MEDIO E PROFISSIONAL em Rua Padre João Antônios/n - Jardim Carvalho, Ponta Grossa - PR, 84015-360
	Nome:  Colégio Estadual Polivalente  
	COD_ID: 345
	URL (copy-paste): http://cepolivalente.wixsite.com/polivalente
	Escola Rating: 4,2 Total Ratings: 27 avaliações
query: COLEGIO ESTADUAL POLIVALENTE DE CANDEIAS em Rua José XavierS/N - Malemba, Candeias - BA, 43805-580
	Nome:  Colégio Estadual Polivalente De Candeias  
	COD_ID: 346
	URL (copy-paste): http://escolas.educacao.ba.gov.br/node/12143
	Escola Rating: 4,0 Total Ratings: 4 avaliações
query: COLÉGIO ESTADUAL POLIVALENTE FREI JOÃO BATISTA em Rua 14380 - Mar

query: COLEGIO ESTADUAL RAIMUNDO ALENCAR LEAO em avenida Jk2747 - setor universitário, Guaraí - TO, 77700-000
/place/ not in url...
	Nome:  Colegio Estadual Raimundo Alencar Leao  
	COD_ID: 370
Exception: can't copy the url
	URL (copy-paste): None
	Escola Rating: 4,0 Total Ratings: 4 avaliações
query: COLEGIO ESTADUAL RAYMUNDO MATTA em Rua Península do JoanesS/N - Lobato, Salvador - BA, 40470-580
	Nome:  Colegio Estadual Raymundo Matta  
	COD_ID: 371
	URL (copy-paste): http://www.salvador.ba.gov.br/
	Repartição pública Rating: 4,4 Total Ratings: 5 avaliações
query: COLEGIO ESTADUAL REITOR EDGARD SANTO em rua eurico guanaess/n - centro, Remanso - BA, 47200-000
	Nome:  Colégio Estadual Reitor Edgard Santos  
	COD_ID: 372
	URL (copy-paste): http://escolas.educacao.ba.gov.br/node/12206
	Escola Rating: 4,8 Total Ratings: 5 avaliações
query: COLÉGIO ESTADUAL RIO BRANCO_ENSINO MÉDIO E PROFISSIONA em Rua Bispo Dom José2426 - Batel, Curitiba - PR, 80440-080
/place/ not in url...
	Nome:  Colégio

...

/place/ not in url...
	Nome:  Rio de Janeiro  
	COD_ID: 1964
Exception: can't copy the end
Exception: can't copy the tel
Exception: can't copy the plus code
Exception: can't copy the url
	URL (copy-paste): None
	None Rating: 4,8 Total Ratings: None
query: UNID ESC MARIA CHAVES em  rua josé bonifácio402 - , Canto do Buriti - PI, 64890-000
/place/ not in url...
	Nome:  Ceep Maria Chaves  
	COD_ID: 1965
Exception: can't copy the url
	URL (copy-paste): None
	None Rating: 5,0 Total Ratings: 4 avaliações
query: UNIDADE DE EDUCAÇÃO BÁSICA FRANCISCO DE ASSIS DO CENTRO DE EDUCAÇÃO BÁSICA FRANCISCO DE ASSIS em SÃO FRANCISCO 509 - , Ijuí - RS, 98700-000
	Nome:  Centro de Educação Básica Francisco de Assis - EFA  
	COD_ID: 1966
	URL (copy-paste): http://www.unijui.edu.br/efa
	Escola de educação para adultos Rating: 4,7 Total Ratings: 14 avaliações
query: UNIDADE DE EDUCAÇÃO PROFISSIONAL DE CURITI em Rua André de Barros750 - , Curitiba - PR, 80010-080
	Nome:  Faculdade Senac Curitiba Centro  
	COD_

query: UNIDADE DE FORMACAO APERF ESPEC EM SAUDE E ESTETICA UNIFAESE  em Rua Felipe Gadelha91 - , São Paulo - SP, 02012-120
/place/ not in url...
	Nome:  UNIFAES - CURSO TÉCNICO EM PRÓTESE DENTÁRIA  
	COD_ID: 1991
Exception: can't copy the url
	URL (copy-paste): None
	Escola técnica Rating: 4,9 Total Ratings: 43 avaliações
query: UNIDADE ESCOLAR ARTUR GONÇALVES DE SOUSA em  avenida raimundo florindo450 - , Lagoa de São Francisco - PI, 64258-000
	Nome:  U E Artur Goncalves De Sousa  
	COD_ID: 1992
Exception: can't copy the url
	URL (copy-paste): None
	Escola Rating: 5,0 Total Ratings: 1 avaliação
query: UNIDADE ESCOLAR BENEDITO MARTINS NAPOLEÃO em  RUA DEPUTADO AFRÂNIO NUNES 201201 - , Barro Duro - PI, 64455-000
	Nome:  U E Benedito Martins Napoleao  
	COD_ID: 1993
Exception: can't copy the url
	URL (copy-paste): None
	Escola Rating: 0,0 Total Ratings: None
query: UNIDADE ESCOLAR COSMA RAMOS DE SOUSA em AVENIDA CORINTO MATOS S/N - , Marcolândia - PI, 64685-000
/place/ not in url...
	Nome

	Nome:  UniRV - Universidade de Rio Verde, Câmpus Caiapônia  
	COD_ID: 2019
	URL (copy-paste): http://www.unirv.edu.br/
	Universidade Rating: 4,7 Total Ratings: 20 avaliações
query: UNIVERSIDADE DE RIO VERDE - UNIRV em Campus universitario fazenda fontes do saber0 - , Rio Verde - GO, 75901-970
/place/ not in url...
	Nome:  Universidade de Rio Verde - UniRV (Bloco V)  
	COD_ID: 2020
	URL (copy-paste): http://www.unirv.edu.br/
	Universidade Rating: 5,0 Total Ratings: 2 avaliações
query: UNIVERSIDADE DE UBERABA - CAMPUS SEDE - UBERABA - 657746X - CAMPUS UBERABA em Av. Nene Sabino, 1801, Santa Marta.1801 - , Uberaba - MG, 38055-550
	Nome:  Uniube - Universidade de Uberaba, Campus Aeroporto  
	COD_ID: 2021
	URL (copy-paste): http://www.uniube.br/
	Universidade particular Rating: 4,4 Total Ratings: 282 avaliações
query: UNIVERSIDADE DO OESTE DE SANTA CATARINA - UNOESC - SEDE em Rua Getúlio Vargas2125 - , Joaçaba - SC, 89600-000
	Nome:  Universidade do Oeste de Santa Catarina, Campus Joaçaba 

	URL (copy-paste): https://www.uninove.br/pos-graduacao/
	Universidade particular Rating: 3,2 Total Ratings: 13 avaliações
query: UNIVERSIDADE NOVE DE JULHO - CAMPUS: UNIDADE VERGUEIRO em Rua Vergueiro235 - , São Paulo - SP, 01504-001
	Nome:  UNINOVE - Campus Vergueiro  
	COD_ID: 2045
	URL (copy-paste): https://www.uninove.br/
	Universidade Rating: 3,8 Total Ratings: 1.745 avaliações
query: UNIVERSIDADE NOVE DE JULHO - CAMPUS: VILA MARIA em Rua Diamantina302 - , São Paulo - SP, 02117-010
	Nome:  UNINOVE - Campus Vila Maria  
	COD_ID: 2046
	URL (copy-paste): http://www.uninove.br/
	Universidade Rating: 3,5 Total Ratings: 714 avaliações
query: UNIVERSIDADE NOVE DE JULHO - VILA PRUDENTE - CAMPUS SÃO PAULO em Avenida Professor Luiz Ignácio Anhaia Mello1363 - , São Paulo - SP, 03155-000
	Nome:  UNINOVE - Campus Vila Prudente  
	COD_ID: 2047
	URL (copy-paste): http://www.uninove.br/
	Universidade Rating: 2,9 Total Ratings: 780 avaliações
query: UNIVERSIDADE NOVE DE JULHO (UNINOVE) - PÓLO BAU

	Nome:  Conservatorio Vinicius de Moraes  
	COD_ID: 2074
Exception: can't copy the url
	URL (copy-paste): None
	Escola de Música Rating: 3,7 Total Ratings: 3 avaliações
query: VIRTUAL EDUCAÇÃO E TECNOLOG em Travessa Quatorze de Março1279 - , Belém - PA, 66055-490
/place/ not in url...
	Nome:  DoGo Maker - Escola de Tecnologia & Inovação  
	COD_ID: 2075
	URL (copy-paste): http://www.dogomaker.com/
	Escola de informática Rating: 5,0 Total Ratings: 9 avaliações
query: VISAO COLEGIO TECNICO EDUCACIONAL em Rua Fernando Camargo611 - , Americana - SP, 13465-020
	Nome:  Colégio Técnico Visão  
	COD_ID: 2076
	URL (copy-paste): http://colegiotecnicovisao.com/
	Escola técnica Rating: 4,4 Total Ratings: 60 avaliações
query: VITAL BRASIL ESCOLA DE FORMACAO PROFISSIONAL em RUA THEREZA MARIA LUIZETTO107 - , Taboão da Serra - SP, 06754-010
	Nome:  Escola Vital  
	COD_ID: 2077
	URL (copy-paste): http://www.escolavital.com.br/
	Escola técnica Rating: 3,5 Total Ratings: 37 avaliações
query: VITÓRIA FORMA